In [1]:
import numpy as np
import pandas as pd
import re
from utils import load_dataset


In [2]:
# Upload datasets

datasets_train, datasets_test, datasets_all = load_dataset()
print('dataset shape', datasets_all.shape)
print('train shape', datasets_train.shape)
print('test shape', datasets_test.shape)

dataset shape (1118, 2)
train shape (895, 2)
test shape (223, 2)


### Text Data Cleaning and Preprocessing

In [3]:
def tokenize_and_normalize(text, remove_small_words = True, leave_only_letters = True, to_lower_case = True):
    """
    Converting a sentence into list of words. Normalize text.
    
    Argument:
    text -- a sentence that should be tokenized and normalized
    to_lower_case -- reduced all words to lowercase. Default value is True
    leave_only_letters -- remove all irrelevant characters (any non-letter characters). Default value is True
    remove_small_words -- remove all small words (less than 3 characters). Default value is True
    
    Returns:
    words -- list of words

    """
    if to_lower_case:
        text=text.lower()
    pattern = r'[A-Z,a-z]' if leave_only_letters else r'\S' 
    pattern += r'{3,}' if remove_small_words else r'{1,}' 
    words=re.findall(pattern,text)
    return words

In [4]:
ds = np.array(datasets_train['message'])

print('!!!!!!!!! sentense datasets_train[1]: ', ds[3])
words = tokenize_and_normalize(ds[3])
print('!!!!!!!!! words: ', words)


!!!!!!!!! sentense datasets_train[1]:  [SPAM]  I will chase you          HealthCare Newsletter        body        font family  Arial  Helvetica  sans serif       margin          padding           a        color       A           noborder         border           noimage     font family Arial  Helvetica  sans serif    font size   px    font weight normal    text align center    text decoration underline     center         text align  center        td permission        font family  Arial  Helvetica  sans serif       font size    px       font weight  normal       color                padding    px     px          td permission a        font family  Arial  Helvetica  sans serif       font size    px       font weight  normal       color                 td header        height    px       border top   px dotted  d d d        border bottom   px dotted  d d d        background color       a         td header h         font family  Arial  Helvetica  sans serif       color  white       font si